# <center>Distribution of Electrical Charging Stations in Bay Area <Center>
<center>by: Su Yu

# 1. Introduction



Since the last few decades, electrical vehicles (EV) have gained an increasing attention and demand as a replacement of traditional fossil fuel-powered vehicles. As of 2018, 45 series production highway-capable all-electric cars are available in various countries, with 300,000 Nissan Leaf and 200,000 Tesla Model S sold across the globe. 

While the EVs are rapidly expanding its business, a major drawback of these vehicles is that their utility relis heavily on the availability of electrical charging stations. The sparse distribution of electrical charging stations in some cities is a primary concern that limits the popularity of EVs.

This project studies the distribution of electrical charging stations in city of San Francisco. This project aims to develop statistical and machine learning methods in analyzing the current distribution of charging stations, and making suggestions to addition of future stations. This project uses geographical data from Foursquare to help develop this analysis. 

The rest of this report is organized as follows. Section 2 describes the data with variable definitions and initial data analysis. Section 3 lists the methods our team used to model this dataset. Section 4 presents theresults of model fitting from each method. Section 5 discusses the analytical findings from the applied methodsand summarizes the outcomes of this class project.

# 2. Data Collection

In this project, Foursquare is used to collect the distribution of electrical charging stations in the city of San Francisco. The collected data will be grouped and clustered geographically to visualize the distribution. The density of the stations at each region is statistically analyzed to reflect the distribution. Finally, suggestions are made based on the density of their distributions in the city. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


## 2.1. Retrieve Data from Foursquare

In [3]:
CLIENT_ID = 'OIK1ZL2VJV5TQ0BZTBVXI5TDIFJUKWXC4QNMRTNBJ0IRZW5I' # your Foursquare ID
CLIENT_SECRET = 'D2G4OIG3EU0NYS11AWM2BOMV5CMDJPRWULOB34YSMRSG4FBY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OIK1ZL2VJV5TQ0BZTBVXI5TDIFJUKWXC4QNMRTNBJ0IRZW5I
CLIENT_SECRET:D2G4OIG3EU0NYS11AWM2BOMV5CMDJPRWULOB34YSMRSG4FBY


In [4]:

address = '3841 Smith St, Union City, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.596383 -122.07800470015084


In [5]:
search_query = 'Electric Charging Station'
radius = 300000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

Electric Charging Station .... OK!


In [6]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.neighborhood
0,586db9f606f1a3416823c94b,Lucky EVgo Charging Station,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",v-1590965803,False,37.534214,-121.996956,"[{'label': 'display', 'lat': 37.534214, 'lng':...",9951,94538,US,Fremont,CA,United States,"[Fremont, CA 94538, United States]",NaN,NaN,NaN
1,50d4a430e4b0c5e2e388b820,Phone Charging Station,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1590965803,False,37.526129,-122.000650,"[{'label': 'display', 'lat': 37.526129, 'lng':...",10380,94560,US,Newark,CA,United States,"[Newark, CA 94560, United States]",NaN,NaN,NaN
2,5dd026cd87fd490007686354,Chargepoint Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.444565,-122.165583,"[{'label': 'display', 'lat': 37.444565, 'lng':...",18585,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN
3,5ad778c81c062d5c5b04ec78,Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.485030,-122.146750,"[{'label': 'display', 'lat': 37.48503, 'lng': ...",13801,94025,US,Menlo Park,CA,United States,"[Menlo Park, CA 94025, United States]",NaN,NaN,NaN
4,5dd024561832c80008886329,Chargepoint Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.446567,-122.162415,"[{'label': 'display', 'lat': 37.446567, 'lng':...",18266,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN
5,5d0acceb22d490002c9238b3,Volta Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.484830,-122.231170,"[{'label': 'display', 'lat': 37.48483, 'lng': ...",18357,94063,US,Redwood City,CA,United States,"[1071 El Camino Real, Redwood City, CA 94063, ...",1071 El Camino Real,NaN,NaN
6,5dd021db18c17100084463f4,SemaConnect Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.445155,-122.161704,"[{'label': 'display', 'lat': 37.445155, 'lng':...",18385,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN
7,5dec3594061a3f0008025e59,EVgo Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.503614,-122.262417,"[{'label': 'display', 'lat': 37.503614, 'lng':...",19275,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN
8,5dec36375da4960008199741,Evgo Charging Station At San Carlos Library,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.503637,-122.262563,"[{'label': 'display', 'lat': 37.503637, 'lng':...",19284,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN
9,5dd01f5518c17100083fb4fa,Chargepoint Charging Station,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",v-1590965803,False,37.443520,-122.163193,"[{'label': 'display', 'lat': 37.44352, 'lng': ...",18604,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN


In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()
# dataframe_filtered.shape

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id
0,Lucky EVgo Charging Station,Automotive Shop,37.534214,-121.996956,"[{'label': 'display', 'lat': 37.534214, 'lng':...",9951,94538,US,Fremont,CA,United States,"[Fremont, CA 94538, United States]",NaN,NaN,NaN,586db9f606f1a3416823c94b
1,Phone Charging Station,Shopping Mall,37.526129,-122.000650,"[{'label': 'display', 'lat': 37.526129, 'lng':...",10380,94560,US,Newark,CA,United States,"[Newark, CA 94560, United States]",NaN,NaN,NaN,50d4a430e4b0c5e2e388b820
2,Chargepoint Charging Station,EV Charging Station,37.444565,-122.165583,"[{'label': 'display', 'lat': 37.444565, 'lng':...",18585,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd026cd87fd490007686354
3,Charging Station,EV Charging Station,37.485030,-122.146750,"[{'label': 'display', 'lat': 37.48503, 'lng': ...",13801,94025,US,Menlo Park,CA,United States,"[Menlo Park, CA 94025, United States]",NaN,NaN,NaN,5ad778c81c062d5c5b04ec78
4,Chargepoint Charging Station,EV Charging Station,37.446567,-122.162415,"[{'label': 'display', 'lat': 37.446567, 'lng':...",18266,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd024561832c80008886329


In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=9) # generate map centred around the Conrad Hotel

#folium.CircleMarker(
#    [latitude, longitude],
#    radius=10,
#    color='red',
#    popup='Twin Peaks, City of San Francisco',
#    fill = True,
#    fill_color = 'red',
#    fill_opacity = 0.6
#).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=1
    ).add_to(venues_map)

# display map
venues_map

## 2.2 Retrieve City Data from Wikipedia

In [9]:
# Scrape table from Wikipedia
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_the_San_Francisco_Bay_Area"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [10]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable plainrowheaders sortable')
# right_table

In [11]:
A=[] # Name of county
B=[] # population 2010
C=[] # sq mi
D=[] # Name of City

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
        A.append(cells[1].find(text=True))
        B.append(cells[2].find(text=True))
        C.append(cells[3].find(text=True))
for row in right_table.findAll('tr'):
    cells=row.findAll('th')
    if len(cells)==1:
        D.append(cells[0].find(text=True))

In [12]:
import pandas as pd
df=pd.DataFrame(D,columns=['City'])
df['County']=A
df['Population']=B
df['LandArea[Sqmi]']=C

df.head()

,City,County,Population,LandArea[Sqmi]
0,Alameda,Alameda,"73,812\n",10.61\n
1,Albany,Alameda,"18,539\n",1.79\n
2,American Canyon,Napa,"19,454\n",4.84\n
3,Antioch,Contra Costa,"102,372\n",28.35\n
4,Atherton,San Mateo,"6,914\n",5.02\n


In [13]:
# Format Dataframe
df.replace('\n','',regex=True,inplace = True)
df.replace(',','',regex=True,inplace = True)
df.iloc[:,2] = df.iloc[:,2].astype('int')
df.iloc[:,3] = df.iloc[:,3].astype('float')

In [14]:
#properly rename dataframes
df_city = df
df_EV = dataframe_filtered

In [15]:
df_EV

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id
0,Lucky EVgo Charging Station,Automotive Shop,37.534214,-121.996956,"[{'label': 'display', 'lat': 37.534214, 'lng':...",9951,94538,US,Fremont,CA,United States,"[Fremont, CA 94538, United States]",NaN,NaN,NaN,586db9f606f1a3416823c94b
1,Phone Charging Station,Shopping Mall,37.526129,-122.000650,"[{'label': 'display', 'lat': 37.526129, 'lng':...",10380,94560,US,Newark,CA,United States,"[Newark, CA 94560, United States]",NaN,NaN,NaN,50d4a430e4b0c5e2e388b820
2,Chargepoint Charging Station,EV Charging Station,37.444565,-122.165583,"[{'label': 'display', 'lat': 37.444565, 'lng':...",18585,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd026cd87fd490007686354
3,Charging Station,EV Charging Station,37.485030,-122.146750,"[{'label': 'display', 'lat': 37.48503, 'lng': ...",13801,94025,US,Menlo Park,CA,United States,"[Menlo Park, CA 94025, United States]",NaN,NaN,NaN,5ad778c81c062d5c5b04ec78
4,Chargepoint Charging Station,EV Charging Station,37.446567,-122.162415,"[{'label': 'display', 'lat': 37.446567, 'lng':...",18266,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd024561832c80008886329
5,Volta Charging Station,EV Charging Station,37.484830,-122.231170,"[{'label': 'display', 'lat': 37.48483, 'lng': ...",18357,94063,US,Redwood City,CA,United States,"[1071 El Camino Real, Redwood City, CA 94063, ...",1071 El Camino Real,NaN,NaN,5d0acceb22d490002c9238b3
6,SemaConnect Charging Station,EV Charging Station,37.445155,-122.161704,"[{'label': 'display', 'lat': 37.445155, 'lng':...",18385,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd021db18c17100084463f4
7,EVgo Charging Station,EV Charging Station,37.503614,-122.262417,"[{'label': 'display', 'lat': 37.503614, 'lng':...",19275,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN,5dec3594061a3f0008025e59
8,Evgo Charging Station At San Carlos Library,EV Charging Station,37.503637,-122.262563,"[{'label': 'display', 'lat': 37.503637, 'lng':...",19284,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN,5dec36375da4960008199741
9,Chargepoint Charging Station,EV Charging Station,37.443520,-122.163193,"[{'label': 'display', 'lat': 37.44352, 'lng': ...",18604,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd01f5518c17100083fb4fa


In [16]:
df_city.head()

,City,County,Population,LandArea[Sqmi]
0,Alameda,Alameda,73812,10.61
1,Albany,Alameda,18539,1.79
2,American Canyon,Napa,19454,4.84
3,Antioch,Contra Costa,102372,28.35
4,Atherton,San Mateo,6914,5.02


In [31]:
df_city

,City,County,Population,LandArea[Sqmi]
0,Alameda,Alameda,73812,10.61
1,Albany,Alameda,18539,1.79
2,American Canyon,Napa,19454,4.84
3,Antioch,Contra Costa,102372,28.35
4,Atherton,San Mateo,6914,5.02
5,Belmont,San Mateo,25835,4.62
6,Belvedere,Marin,2068,0.52
7,Benicia,Solano,26997,12.93
8,Berkeley,Alameda,112580,10.47
9,Brentwood,Contra Costa,51481,14.79


In [17]:
df_EV_group=df_EV.groupby('city')['name'].count()

In [18]:
df = pd.merge(df_city,df_EV_group,how = 'left',left_on='City',right_on = 'city')

In [19]:
df.rename(columns = {'name':'EVCS Count'}, inplace = True)

In [20]:
df.fillna(0,inplace = True)

In [21]:
df_county = df.groupby('County',as_index=False).sum()

In [22]:
df_county['EVCS-Pop Ratio']=df_county['EVCS Count']/df_county['Population']
x = df_county['EVCS-Pop Ratio'].values
norm1 = x / max(x)
df_county['Normalized EVCS-Pop Ratio']=norm1
df_county.drop(columns = 'EVCS-Pop Ratio',inplace = True)
df_county.sort_values(by='Normalized EVCS-Pop Ratio',ascending = False, inplace = True)
df_county.loc[:,'EVCS Count']=df_county['EVCS Count'].astype(int)

In [23]:
df_county['EVCS-Land Ratio']=df_county['EVCS Count']/df_county['LandArea[Sqmi]']
x = df_county['EVCS-Land Ratio'].values
norm1 = x / max(x)
df_county['Normalized EVCS-Land Ratio']=norm1
df_county.drop(columns = 'EVCS-Land Ratio',inplace = True)

In [24]:
df_county.reset_index(inplace = True, drop = True)
df_county

,County,Population,LandArea[Sqmi],EVCS Count,Normalized EVCS-Pop Ratio,Normalized EVCS-Land Ratio
0,Marin,184982,68.02,3,1.000000,0.792947
1,San Mateo,657229,143.83,8,0.750553,1.000000
2,Santa Clara,1691682,352.86,15,0.546740,0.764273
3,Sonoma,348463,83.73,2,0.353901,0.429446
4,Solano,394510,127.16,2,0.312594,0.282774
5,Contra Costa,889240,270.90,3,0.208023,0.199100
6,Alameda,1369005,315.24,3,0.135121,0.171096
7,San Francisco,805235,46.87,1,0.076575,0.383588
8,Napa,110271,31.80,0,0.000000,0.000000


In [25]:
bay_geo = r'california-counties.geojson'

with open(bay_geo) as bay_geo_file:
    bay_geo_json = json.load(bay_geo_file)

In [26]:
denominations_json = []
for index in range(len(bay_geo_json['features'])):
    denominations_json.append(bay_geo_json['features'][index]['properties']['name'])
for county in df_county['County']:
    print(county in denominations_json)

True
True
True
True
True
True
True
True
True


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=1
    ).add_to(bay_map)

In [27]:
# Use K-means to group
df_all = pd.merge(df_EV,df_city,how = 'left',left_on='city',right_on = 'City',copy=False)
df_all

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id,City,County,Population,LandArea[Sqmi]
0,Lucky EVgo Charging Station,Automotive Shop,37.534214,-121.996956,"[{'label': 'display', 'lat': 37.534214, 'lng':...",9951,94538,US,Fremont,CA,United States,"[Fremont, CA 94538, United States]",NaN,NaN,NaN,586db9f606f1a3416823c94b,Fremont,Alameda,214089.0,77.46
1,Phone Charging Station,Shopping Mall,37.526129,-122.000650,"[{'label': 'display', 'lat': 37.526129, 'lng':...",10380,94560,US,Newark,CA,United States,"[Newark, CA 94560, United States]",NaN,NaN,NaN,50d4a430e4b0c5e2e388b820,Newark,Alameda,42573.0,13.87
2,Chargepoint Charging Station,EV Charging Station,37.444565,-122.165583,"[{'label': 'display', 'lat': 37.444565, 'lng':...",18585,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd026cd87fd490007686354,Palo Alto,Santa Clara,64403.0,23.88
3,Charging Station,EV Charging Station,37.485030,-122.146750,"[{'label': 'display', 'lat': 37.48503, 'lng': ...",13801,94025,US,Menlo Park,CA,United States,"[Menlo Park, CA 94025, United States]",NaN,NaN,NaN,5ad778c81c062d5c5b04ec78,Menlo Park,San Mateo,32026.0,9.79
4,Chargepoint Charging Station,EV Charging Station,37.446567,-122.162415,"[{'label': 'display', 'lat': 37.446567, 'lng':...",18266,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd024561832c80008886329,Palo Alto,Santa Clara,64403.0,23.88
5,Volta Charging Station,EV Charging Station,37.484830,-122.231170,"[{'label': 'display', 'lat': 37.48483, 'lng': ...",18357,94063,US,Redwood City,CA,United States,"[1071 El Camino Real, Redwood City, CA 94063, ...",1071 El Camino Real,NaN,NaN,5d0acceb22d490002c9238b3,Redwood City,San Mateo,76815.0,19.42
6,SemaConnect Charging Station,EV Charging Station,37.445155,-122.161704,"[{'label': 'display', 'lat': 37.445155, 'lng':...",18385,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd021db18c17100084463f4,Palo Alto,Santa Clara,64403.0,23.88
7,EVgo Charging Station,EV Charging Station,37.503614,-122.262417,"[{'label': 'display', 'lat': 37.503614, 'lng':...",19275,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN,5dec3594061a3f0008025e59,San Carlos,San Mateo,28406.0,5.54
8,Evgo Charging Station At San Carlos Library,EV Charging Station,37.503637,-122.262563,"[{'label': 'display', 'lat': 37.503637, 'lng':...",19284,94070,US,San Carlos,CA,United States,"[610 Elm St, San Carlos, CA 94070, United States]",610 Elm St,NaN,NaN,5dec36375da4960008199741,San Carlos,San Mateo,28406.0,5.54
9,Chargepoint Charging Station,EV Charging Station,37.443520,-122.163193,"[{'label': 'display', 'lat': 37.44352, 'lng': ...",18604,94301,US,Palo Alto,CA,United States,"[Palo Alto, CA 94301, United States]",NaN,NaN,NaN,5dd01f5518c17100083fb4fa,Palo Alto,Santa Clara,64403.0,23.88


In [28]:
df_all = df_all.loc[:,['name',
                          'categories',
                          'lat','lng',
                          'City',
                          'County',
                          'Population',
                          'LandArea[Sqmi]']]
df_all.dropna(inplace = True)
df_all.reset_index(inplace = True, drop = True)
df_all.head()

,name,categories,lat,lng,City,County,Population,LandArea[Sqmi]
0,Lucky EVgo Charging Station,Automotive Shop,37.534214,-121.996956,Fremont,Alameda,214089.0,77.46
1,Phone Charging Station,Shopping Mall,37.526129,-122.000650,Newark,Alameda,42573.0,13.87
2,Chargepoint Charging Station,EV Charging Station,37.444565,-122.165583,Palo Alto,Santa Clara,64403.0,23.88
3,Charging Station,EV Charging Station,37.485030,-122.146750,Menlo Park,San Mateo,32026.0,9.79
4,Chargepoint Charging Station,EV Charging Station,37.446567,-122.162415,Palo Alto,Santa Clara,64403.0,23.88


In [29]:
bay_map = folium.Map(location=[latitude, longitude], zoom_start=8)
bay_map.choropleth(
    geo_data = bay_geo_json,
    data = df_county,
    columns = ['County','Normalized EVCS-Pop Ratio'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrRd', 
    fill_opacity=0.6, 
    line_opacity=0.5,
    #legend_name='Normalized EV Charging Station Count vs. County Population'
)
from folium import plugins
incidents = plugins.MarkerCluster().add_to(bay_map)
for lat, lng, label, in zip(df_all.lat, df_all.lng, df_all.County):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)
bay_map

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [30]:
bay_map = folium.Map(location=[latitude, longitude], zoom_start=8)
bay_map.choropleth(
    geo_data = bay_geo_json,
    data = df_county,
    columns = ['County','Normalized EVCS-Land Ratio'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrRd', 
    fill_opacity=0.6, 
    line_opacity=0.5,
    #legend_name='Normalized EV Charging Station Count vs. County Population'
)
from folium import plugins
incidents = plugins.MarkerCluster().add_to(bay_map)
for lat, lng, label, in zip(df_all.lat, df_all.lng, df_all.County):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)
bay_map